In [1]:
from datasets import Dataset

dataset = Dataset.load_from_disk("testcase_UniRef50")

In [2]:
print(dataset)

Dataset({
    features: ['UniRef50_O59651', 'UniRef50_P0A300', 'UniRef50_A0A651EUH4', 'UniRef50_P28371', 'UniRef50_P37744', 'UniRef50_A0A0F7KVE1', 'UniRef50_A0A645ERY2', 'UniRef50_P11044', 'UniRef50_P39751', 'UniRef50_P69806', 'UniRef50_A0A0T5P7Z0', 'UniRef50_P56933', 'UniRef50_A0A848MHV8', 'UniRef50_A0A7W5ZU98', 'UniRef50_A0A6L4YIC8', 'UniRef50_P43890', 'UniRef50_A0A174PWJ1', 'UniRef50_A0A6J4FJ68', 'UniRef50_P0DF94', 'UniRef50_O87388', 'UniRef50_A0A132BTP8', 'UniRef50_A0A024H9T2', 'UniRef50_P94551', 'UniRef50_P55253', 'UniRef50_A0A162RGU2', 'UniRef50_A0A0T5P760', 'UniRef50_P62656', 'UniRef50_A0A031G870', 'UniRef50_P77249', 'UniRef50_A0A0X8X7E8', 'UniRef50_P44516', 'UniRef50_P52276', 'UniRef50_A0A1B1ULJ5', 'UniRef50_P37949', 'UniRef50_A0A0N8K4N6', 'UniRef50_A0A653ZXD6', 'UniRef50_A0A011Q4Z1', 'UniRef50_A0A0M3Q136', 'UniRef50_A0A0F5Q329', 'UniRef50_A0A5Q0JW75', 'UniRef50_A0A0P0DEX9', 'UniRef50_A0A0P9TWJ2', 'UniRef50_P11557', 'UniRef50_A0A812M8M9', 'UniRef50_O67618', 'UniRef50_A0A9D2D933